In [3]:
# 生成train_data12.csv
# 添加规则匹配的训练集订单（训练集路径包含测试集路径）
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

def process_order(df):
    order_df = pd.read_csv( '../../../dataset/order_data/B_order_info.csv')
    trace_df = order_df.drop_duplicates(['TRANSPORT_TRACE']).reset_index(drop = True)
    
    df = df.merge(trace_df[['TRANSPORT_TRACE', 'start_lon', 'start_lat', 'end_lon', 'end_lat']], on = 'TRANSPORT_TRACE', how = 'left')
    
    train_data = pd.DataFrame()
    orders = df.loadingOrder.unique()
    print('orders num:{}'.format(len(orders)))
    for i in range(len(orders)):
        order_df = df[df['loadingOrder'] == orders[i]].reset_index(drop = True)
        start_lon = order_df.loc[0, 'start_lon']
        start_lat = order_df.loc[0, 'start_lat']
        
        end_lon = order_df.loc[0, 'end_lon']
        end_lat = order_df.loc[0, 'end_lat']
        
#         print('\nstart_lon:{}\tstart_lat:{}'.format(start_lon, start_lat))
#         print('end_lon:{}\tend_lat:{}'.format(end_lon, end_lat))
        
        dis_start, dis_end = [], []
        for j in range(len(order_df)):
            dis1 = abs(order_df.loc[j, 'longitude'] - start_lon) + abs(order_df.loc[j, 'latitude'] - start_lat)
            dis2 = abs(order_df.loc[j, 'longitude'] - end_lon) + abs(order_df.loc[j, 'latitude'] - end_lat)
        
            dis_start.append(dis1)
            dis_end.append(dis2)
        
        index_start = dis_start.index(min(dis_start))
        index_end = dis_end.index(min(dis_end))
        
        print('order{}:{} trace:{}\t start:{}\tend:{}'.format(i, orders[i], order_df.loc[0, 'TRANSPORT_TRACE'], index_start, index_end))
        
        if (index_end - index_start > 2):
            order_df = order_df.iloc[index_start:index_end + 1, :].reset_index(drop = True)
            train_data = train_data.append(order_df, ignore_index = True)
    
    train_data = train_data.drop(['start_lon', 'start_lat', 'end_lon', 'end_lat'], axis = 1)
    train_data = train_data.sort_values(['loadingOrder', 'timestamp'], ascending = True).reset_index(drop = True)
    return train_data


# 将匹配的重复训练集订单更名添加
def get_match_df(train_df, match_order_df):
    train_data = pd.DataFrame()
    for i in range(len(match_order_df)):
        print('\ni:{}'.format(i))
        match_order = match_order_df.loc[i, 'match_train_order']
        
        # 训练集数据
        match_df = train_df[train_df['loadingOrder'] == match_order] .reset_index(drop = True)
        match_df['loadingOrder'] = match_order + '_' + '{}'.format(i)
        match_df['TRANSPORT_TRACE'] =  match_order_df.loc[i, 'TRANSPORT_TRACE']
 
        print(match_order_df.loc[i, 'TRANSPORT_TRACE'])
        print(match_df.loc[0,'TRANSPORT_TRACE'])

    
        train_data = train_data.append(match_df, ignore_index = True)

    train_data = train_data[train_df.columns]
    train_data = train_data.sort_values(['loadingOrder', 'timestamp'], ascending = True)
    
    return train_data


start = time.process_time()
match_order_df = pd.read_csv( '../../../dataset/test_data/match_order_v2.csv')
match_orders = match_order_df.match_train_order.unique()

add_df = pd.DataFrame()
for i in range(10):
    print('i:{}'.format(i))
    train_df = pd.read_csv('../../../dataset/chunk_data/train_data{}.csv'.format(i), header = None)
    train_df.columns = ['loadingOrder', 'carrierName', 'timestamp', 'longitude',
                          'latitude', 'vesselMMSI', 'speed', 'direction', 'vesselNextport',
                          'vesselNextportETA', 'vesselStatus', 'vesselDatasource', 'TRANSPORT_TRACE']
    print(train_df.loc[len(train_df)-1, 'loadingOrder'])
    
    train_df = train_df.loc[(train_df['loadingOrder'].isin (match_orders))]
    train_df = train_df[(train_df['direction'] <= 36000) & (train_df['speed'] < 60)].reset_index(drop = True)
    
    add_df = add_df.append(train_df, ignore_index = True)
    print(len(train_df))
    print('read csv: {} seconds'.format(time.process_time() - start))

add_df = add_df[train_df.columns]
add_df = add_df.sort_values(['loadingOrder', 'timestamp'], ascending = True)

add_df = get_match_df(add_df, match_order_df)

# 截取匹配路径
match_df = process_order(add_df)


order = match_df.loadingOrder.unique()
print(order)
print(len(order))

match_df.to_csv('../../../dataset/train_data/train_wash12.csv', index = False, header = None)
print('running time: {} seconds'.format(time.process_time() - start))

i:0
CX425495580514
149736
read csv: 22.243905566999956 seconds
i:1
FK857864540293
157262
read csv: 43.409585507999964 seconds
i:2
HW907489849828
86546
read csv: 64.530836368 seconds
i:3
KP455371889804
116665
read csv: 84.88279331299998 seconds
i:4
NF901306911140
156037
read csv: 105.76191965499999 seconds
i:5
PV603106113253
156253
read csv: 126.35723700899996 seconds
i:6
SM861453907016
117707
read csv: 146.530976269 seconds
i:7
VC489290307158
92317
read csv: 167.62590648599996 seconds
i:8
XV497928381083
124527
read csv: 188.593023469 seconds
i:9
ZZ992682575994
123065
read csv: 208.80159274799996 seconds

i:0
CNSHK-IDJKT
CNSHK-IDJKT

i:1
CNYTN-SGSIN-MTMLA-DZALG
CNYTN-SGSIN-MTMLA-DZALG

i:2
CNNSA-SGSIN-AEJEA
CNNSA-SGSIN-AEJEA

i:3
CNYTN-MXZLO
CNYTN-MXZLO

i:4
CNYTN-PAONX
CNYTN-PAONX

i:5
CNSHK-SGSIN
CNSHK-SGSIN

i:6
CNYTN-GBFXT
CNYTN-GBFXT

i:7
CNSHK-SGSIN-MYPKG-INMUN-PKKHI-PKBQM
CNSHK-SGSIN-MYPKG-INMUN-PKKHI-PKBQM

i:8
CNYTN-SGSIN-EGSUZ-GRPIR-ITSPE-ITGOA
CNYTN-SGSIN-EGSUZ-GRPIR-ITSPE-IT

order43:GT324754989864_33 trace:HKHKG-FRFOS	 start:3	end:3297
order44:GW888707016738_107 trace:CNSHK-SGSIN	 start:0	end:735
order45:GX775979109232_141 trace:CNYTN-PAONX	 start:4	end:5624
order46:HM832145179255_47 trace:CNYTN-MXZLO	 start:125	end:4447
order47:HQ550132211431_70 trace:CNSHK-MYTPP	 start:0	end:4398
order48:HX961597046747_95 trace:CNSHK-SGSIN	 start:0	end:3349
order49:IE974931196043_13 trace:CNSHK-SGSIN	 start:0	end:3789
order50:IF896260473107_32 trace:CNSHK-CLVAP	 start:10	end:5482
order51:IJ816138076119_88 trace:CNYTN-SGSIN	 start:0	end:874
order52:IL529259468168_128 trace:CNYTN-MXZLO	 start:0	end:32372
order53:IO982928181834_49 trace:CNSHK-IDJKT	 start:0	end:1129
order54:IT177474847833_11 trace:CNSHK-ZADUR	 start:0	end:1735
order55:JH703731803429_78 trace:CNNSA-SGSIN-AEJEA	 start:0	end:1177
order56:JJ150955255913_145 trace:CNSHK-SGSIN-MYPKG	 start:0	end:621
order57:JY553290031778_46 trace:CNHKG-RUVVO	 start:3	end:4349
order58:JZ400494084655_127 trace:CNYTN-CAVAN	 start:1

running time: 340.705589835 seconds
